## Travelling Salesman problem

In [60]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

In [61]:
!pip install ortools

In [94]:

from IPython.display import display, HTML
from dimod import BinaryQuadraticModel
import random
import numpy as np
import os
import itertools
import time
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

random.seed(42)
cities = ["Munich", "Hamburg", "Stuttgart", "Berlin", "Frankfurt", "Paris", "Madrid", "Barcelona"]
n = len(cities)
A = 10
B = 10
C = 1
distances = {}
for cindex1 in range(len(cities)-1):
    c1 = cities[cindex1]
    for cindex2 in range(cindex1+1,len(cities)):
        c2 = cities[cindex2]
        distances[c1,c2] = random.randint(1,6)
        distances[c2,c1] = distances[c1,c2]

print(distances)

distance_matrix = np.zeros((n, n), dtype=int)
for (c1, c2), dist in distances.items():
    i, j = cities.index(c1), cities.index(c2)
    distance_matrix[i, j] = distance_matrix[j, i] = dist

print("Distance Matrix:")
print(distance_matrix)


{('Munich', 'Hamburg'): 6, ('Hamburg', 'Munich'): 6, ('Munich', 'Stuttgart'): 1, ('Stuttgart', 'Munich'): 1, ('Munich', 'Berlin'): 1, ('Berlin', 'Munich'): 1, ('Munich', 'Frankfurt'): 6, ('Frankfurt', 'Munich'): 6, ('Munich', 'Paris'): 3, ('Paris', 'Munich'): 3, ('Munich', 'Madrid'): 2, ('Madrid', 'Munich'): 2, ('Munich', 'Barcelona'): 2, ('Barcelona', 'Munich'): 2, ('Hamburg', 'Stuttgart'): 2, ('Stuttgart', 'Hamburg'): 2, ('Hamburg', 'Berlin'): 6, ('Berlin', 'Hamburg'): 6, ('Hamburg', 'Frankfurt'): 1, ('Frankfurt', 'Hamburg'): 1, ('Hamburg', 'Paris'): 6, ('Paris', 'Hamburg'): 6, ('Hamburg', 'Madrid'): 6, ('Madrid', 'Hamburg'): 6, ('Hamburg', 'Barcelona'): 5, ('Barcelona', 'Hamburg'): 5, ('Stuttgart', 'Berlin'): 1, ('Berlin', 'Stuttgart'): 1, ('Stuttgart', 'Frankfurt'): 5, ('Frankfurt', 'Stuttgart'): 5, ('Stuttgart', 'Paris'): 4, ('Paris', 'Stuttgart'): 4, ('Stuttgart', 'Madrid'): 1, ('Madrid', 'Stuttgart'): 1, ('Stuttgart', 'Barcelona'): 1, ('Barcelona', 'Stuttgart'): 1, ('Berlin', 'F

### Creates el data model

In [92]:
from ortools.constraint_solver import pywrapcp
from ortools.sat.python import cp_model

# Definir el modelo de OR-Tools
model = cp_model.CpModel()

# Variables: x[t][c] es 1 si la ciudad c es visitada en el tiempo t
x = {}
for t in range(n):
    for c in range(n):
        x[t, c] = model.NewBoolVar(f'x[{t},{c}]')

# Restricción 1: cada ciudad debe ser visitada exactamente una vez
for c in range(n):
    model.Add(sum(x[t, c] for t in range(n)) == 1)

# Restricción 2: cada tiempo debe tener exactamente una ciudad
for t in range(n):
    model.Add(sum(x[t, c] for c in range(n)) == 1)


# Función objetivo: minimizar la distancia total recorrida
total_distance = model.NewIntVar(0, 10000, 'total_distance')

# Crear una lista de términos a sumar
terms = []
for t in range(n - 1):
    for c1 in range(n):
        for c2 in range(n):
            term = model.NewIntVar(0, 10000, f'term[{t},{c1},{c2}]')
            model.AddMultiplicationEquality(term, [distance_matrix[c1, c2], x[t, c1], x[t + 1, c2]])
            terms.append(term)

# Añadir el término de la vuelta al inicio
for c1 in range(n):
    for c2 in range(n):
        term = model.NewIntVar(0, 10000, f'term_end[{c1},{c2}]')
        model.AddMultiplicationEquality(term, [distance_matrix[c2, c1], x[n - 1, c1], x[0, c2]])
        terms.append(term)

# Sumar todos los términos para definir la función objetivo
model.Add(total_distance == sum(terms))
model.Minimize(total_distance)


### Find the minima usiNG ROUTINGModel

In [93]:
import time
import psutil 
import numpy as np

def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024  # Convertir a MB

# Resolver el problema
solver = cp_model.CpSolver()
start_time = time.time()
initial_memory = get_memory_usage()
status = solver.Solve(model)
end_time = time.time()
final_memory = get_memory_usage()
execution_time = end_time - start_time

# Mostrar resultados
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    route = [-1] * n
    for t in range(n):
        for c in range(n):
            if solver.BooleanValue(x[t, c]):
                route[t] = c
    total_distance_value = solver.Value(total_distance)
    
    print("La ruta óptima es:", [cities[i] for i in route])
    print("Distancia total recorrida:", total_distance_value)
    print("Tiempo de ejecución:", execution_time, "segundos")

else:
    print("No se encontró una solución óptima.")

# Construir la matriz resultado para visualizar la ruta
result = np.zeros((n, n), dtype=object)
for t in range(n):
    for c in range(n):
        if solver.BooleanValue(x[t, c]):
            result[t, c] = cities[c]
        else:
            result[t, c] = 0
print("Incremento de uso de memoria:", final_memory - initial_memory, "MB")
print("CPU Solver Formatted Result:")
for row in result:
    print(row)

La ruta óptima es: ['Munich', 'Barcelona', 'Stuttgart', 'Hamburg', 'Frankfurt', 'Madrid', 'Paris', 'Berlin']
Distancia total recorrida: 12
Tiempo de ejecución: 0.08057403564453125 segundos
Incremento de uso de memoria: 1.14453125 MB
CPU Solver Formatted Result:
['Munich' 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 'Barcelona']
[0 0 'Stuttgart' 0 0 0 0 0]
[0 'Hamburg' 0 0 0 0 0 0]
[0 0 0 0 'Frankfurt' 0 0 0]
[0 0 0 0 0 0 'Madrid' 0]
[0 0 0 0 0 'Paris' 0 0]
[0 0 0 'Berlin' 0 0 0 0]


### Draw Solution

In [41]:
import matplotlib.pyplot as plt
import numpy as np

# Función para visualizar la ruta y las distancias entre ciudades
def plot_route(route, cities, distance_matrix):
    n = len(cities)
    plt.figure(figsize=(10, 10))  # Aumentar el tamaño del gráfico
    ax = plt.gca()

    # Coordenadas de las ciudades (para visualización)
    coords = np.random.rand(n, 2) * 100  # Coordenadas aleatorias en un plano 100x100
    for i, (x, y) in enumerate(coords):
        ax.plot(x, y, 'o', markersize=10, label=cities[i])
        ax.text(x, y, f' {cities[i]}', fontsize=12)

    # Dibujar la ruta y añadir las distancias
    total_distance = 0
    for i in range(n):
        start_city = route[i]
        end_city = route[(i + 1) % n]
        start_coord = coords[start_city]
        end_coord = coords[end_city]
        ax.plot([start_coord[0], end_coord[0]], [start_coord[1], end_coord[1]], 'k-')
        
        # Calcular el punto medio para poner la etiqueta de la distancia
        mid_point = (start_coord + end_coord) / 2
        distance = distance_matrix[start_city, end_city]
        total_distance += distance
        
        # Ajustar posición de la etiqueta para que no se solape
        label_offset = np.array([0, 0])  # Ajuste vertical
        if start_coord[0] > end_coord[0]:
            label_offset[0] = -15  # Ajuste horizontal si necesario
        ax.text(mid_point[0] + label_offset[0], mid_point[1] + label_offset[1], 
                f'{distance}', color='red', fontsize=10, ha='center')

    # Dibujar la línea de regreso a la ciudad inicial
    start_city = route[0]
    start_coord = coords[start_city]
    end_coord = coords[route[-1]]
    ax.plot([start_coord[0], end_coord[0]], [start_coord[1], end_coord[1]], 'k-')

    plt.title(f"Ruta Óptima del Viajante - Distancia Total: {total_distance}")
    plt.legend()
    plt.grid(True)
    plt.show()

# Llamar a la función para dibujar la ruta con distancias
plot_route(sorted_route, cities, distance_matrix)


NameError: name 'sorted_route' is not defined